--- 
# Model Fitting and Techniques 
***

The overall goal of this section is to try various techniques to fit a model for mortality rate using food consumption data.  First, we will find a null model, representing the 'average' input and representing a baseline estimation that we will then improve upon. Then we will fit a multilinear regression to all of the predictors (all livestock and all crop predictors), and find a cross-validated $R^2$ for this naive model. Next, we will try more advanced techniques such as Lasso, Ridge, Step-wise, and Regression Trees to improve this model. 

To summarize, our null model achieved a cross-validated $R^2$ score of 0 for all three diseases. Our naive model achieved a cross-validated score of $ $ for diabetes, $ $ for cancer, and $ $ for cardiovascular diseases.

## Null Model

Before fitting the linear regression, we will find a simple null model for global food consumption data. To calculate the null model, we found the average of each predictor column in the Dataframe. This gives us a 'global average' of consumption of each predictor. We can then use the null model to establish a baseline $R^2$ that we will then improve upon using our linear regression models.

In [20]:
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as colors
import pandas as pd
import math
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.cross_validation import train_test_split as sk_split
import statsmodels.api as sm

%matplotlib inline

In [9]:
# Read in initial dataframe
x_df = pd.read_csv('datasets/predictors_filled.csv')

# read in disease rates
diabetes_df = pd.read_csv('datasets/diabetes_df.csv',index_col = 0)
cardio_df = pd.read_csv('datasets/cardio_df.csv',index_col = 0)
cancer_df= pd.read_csv('datasets/cancer_df.csv',index_col = 0)

### Null Model testing:

As expected, testing the null model on various training set give us a cross-validated $R^2$ of approximately zero for all three diseases. 

#### Cancer: 
The null model for cancer will always predict the mean cancer mortality rate. Testing on cancer, we get an $R^2$ of 3.33 E -16, which is ~ 0:

In [12]:
# Null Model Cancer
null_model = LinReg()
null_model.fit(x_df, [np.mean(cancer_df['Cancer Mortality Rate'])]*x_df.shape[0])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [13]:
# Test Cancer.
null_model.score(x_df, cancer_df)

3.3306690738754696e-16

#### Diabetes
Testing on diabetes, we also get an $R^2$ of 0.

In [14]:
# Fit Diabetes Null Model
null_model = LinReg()
null_model.fit(x_df, [np.mean(diabetes_df['Diabetes Mortality Rate'])]*x_df.shape[0])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [ ]:
# Test Diabetes.
null_model.score(x_df, diabetes_df)

#### Test Cardiovascular Diseases

In [16]:
# Test cardiovascular diseases
null_model = LinReg()
null_model.fit(x_df, [np.mean(cardio_df['Cardio Mortality Rate'])]*x_df.shape[0])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [18]:
# Test cardiovascular diseases.
null_model.score(x_df, cardio_df)

0.0

# Simple LinReg

## Cancer LinReg

Now, we will fit a simple multi-linear regression to all of the food consumption inputs for each of the diseases. First, for cancer, our regression has an initial $R^2$ on the training set of .85, and a cross-validated $R^2$ of -16.5 for $k = 5$ folds. 

In [52]:
linreg = LinReg()
linreg.fit(x_df, cancer_df)
print "Training r^2:",linreg.score(x_df, cancer_df)

Training r^2: 0.858582674822


In [51]:
# Cross validated R-squared score
np.mean(cross_val_score(LinReg(), x_df,cancer_df, cv = KFold(151, 5), scoring = "r2"))

-16.502846796333834

To further examine the accuracy of this model, the map below displays the fractional difference of the model estimates as compared to the actual cancer data on a world map. As we can see, the vast majority of countries are colored a dark blue/ purple color, indicating they have a low fractional difference. Countries colored a brighter purple/pink color indicate an overestimate, while countries colored in a brighter blues indicate an underestimate.

In [11]:
# PUT GRAPH HERE

## Diabetes LinReg
For diabetes, our regression has an initial cross-validated $R^2$ of ** PUT THE R^2 HERE**. 

In [73]:
linreg = LinReg()
linreg.fit(x_df, diabetes_df)
print "Training R^2", linreg.score(x_df, diabetes_df)

# Cross-validated R^2 score for diabetes
print "CV R^2 score:",np.mean(cross_val_score(LinReg(), x_df,diabetes_df, cv = KFold(5), scoring = "r2"))

Training R^2 0.834368749536
CV R^2 score: -4.55361320841


Again, we can examine a world map to see the fractional differences. Again, dark blue/purple colors indicate an accurate estimate,brighter purples/pinks indicate an overestimate, and brighter blues indicate an underestimate. We note thatthis model has slightly worse performance than our cancer model, which may have to do with the fact that diabetes does not lead to death as commonly as cancer does.

## Cardiovascular Diseases LinReg
For diabetes, our regression has an initial $R^2$ on the training set of .856, and a cross-validated $R^2$ of -6.02

In [74]:
linreg = LinReg()
linreg.fit(x_df, cardio_df)
print "Training R^2", linreg.score(x_df, cardio_df)
print "Cross-validated R^2", np.mean(cross_val_score(LinReg(), x_df,cardio_df, cv = KFold(151, 5), scoring = "r2"))

Training R^2 0.856203051095
Cross-validated R^2 -6.02431649487


Again, we can examine a world map to see the fractional differences. Dark blue/purple colors indicate an accurate estimate,brighter purples/pinks indicate an overestimate, and brighter blues indicate an underestimate. While this model appears to be fairly accurate for *THESE COUNTRIES*, it could be improved for *THESE*. This might be due to certain predictors, such as *SOME RANDOM PREDICTOR*, that is more heavily weighted for larger countries than for the country that is seeing a larger fractional difference.

--- 
# Advanced Models 
***

In this section, we will use various other regression techniques and variable selection techniques to attempt to improve upon our naive model. In particular, we will try 

1. Lasso
2. PCA
3. Regression Tree
4. Step-wise Variable Selection

For reference, our naive model gives us the following cross-validated $R^2$ values with $k = 5$: 

||Cardio | Diabetes | Cancer
|--- | --- | --- | ---|
|R^2 (Training)| .856 | .834 | .856|

## Lasso

The naive model brought up in the previous section has one major flaw: by including all of the predictors, it is very likely to be overfitted to the initial dataset. As such, we would like to reduce that overfitting by using variable selection techniques such as Lasso to reduce the number of predictors our model includes. 
Using the LassoCV package in sklearn, we obtain the following cross-validated $R^2$:

| |Cardio   |  Diabetes | Cancer  |
|-----|---|---|---|
|$r^2$ (Lasso) |  .459 |  .297 |   .059|


Below is a function we used to calculate the cross-validated r^2 for lasso over a number of folds for a certain parameter value.

In [82]:
def lasso_k_fold_r_squared(x_train, y_train, num_folds, param_val):
    n_train = x_train.shape[0]
    n = int(np.round(n_train * 1. / num_folds)) # points per fold

    # Iterate over folds
    cv_r_squared = 0
    
    for fold in range(1, num_folds + 1):
        # Take k-1 folds for training 
        x_first_half = x_train.iloc[:n * (fold - 1), :]
        x_second_half = x_train.iloc[n * fold + 1:, :]
        x_train_cv = np.concatenate((x_first_half, x_second_half), axis=0)
        
        y_first_half = y_train.iloc[:n * (fold - 1)]
        y_second_half = y_train.iloc[n * fold + 1:]
        y_train_cv = np.concatenate((y_first_half, y_second_half), axis=0)
        
        # Take the middle fold for testing
        x_test_cv = x_train.iloc[1 + n * (fold - 1):n * fold, :]
        y_test_cv = y_train.iloc[1 + n * (fold - 1):n * fold]

        # Fit Decision Tree model with parameter value on CV train set, and evaluate CV test performance
        reg = Lasso(alpha = param_val, normalize=True)
        reg.fit(x_train_cv, y_train_cv)
        coefficients = reg.coef_
        #print len([i for i, item in enumerate(coefficients) if abs(item) >0])
        r_squared = reg.score(x_test_cv, y_test_cv)
    
        # Cummulative R^2 value across folds
        cv_r_squared += r_squared

    # Return average R^2 value across folds
    return cv_r_squared * 1.0 / num_folds

### Cancer

Running Lasso on different values of alpha yields a top cross-validated $r^2$ score of .059, for alpha = .1

In [91]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

print "LASSO"
for alpha in [.001, .01, .1, 1,10,100]:
    print "cancer",alpha, lasso_k_fold_r_squared(x_df,cancer_df,5, alpha)

LASSO
cancer 0.001 -8.74966599085
cancer 0.01 -2.96325751054
cancer 0.1 -0.190699091902
cancer 1 0.059000481265
cancer 10 -0.0667064097657
cancer 100 -0.0667064097657


### Cardiovascular Diseases
Running Lasso on different values of alpha yields a top cross-validated $r^2$ score of .459, for alpha = .9

In [96]:
print "LASSO"
for alpha in [.001, .1, .5,.8, .9, 1, 2,5, 10, 100, 1000]:
    print "cardio",alpha, lasso_k_fold_r_squared(x_df,cardio_df,5, alpha)
    

LASSO
cardio 0.001 -4.38550193519
cardio 0.1 -0.123533778277
cardio 0.5 0.425219743525
cardio 0.8 0.456605362725
cardio 0.9 0.459219427244
cardio 1 0.45802637765
cardio 2 0.410695165821
cardio 5 0.0288024996148
cardio 10 -0.0121108016136
cardio 100 -0.0121108016136
cardio 1000 -0.0121108016136


### Diabetes 
Running Lasso on different values of alpha from .001 to 100 yields a top cross-validated $r^2$ score of .29, for alpha = .1

In [106]:
print "LASSO"
for alpha in [.001, .01, .05, .1, .5, 1, 10, 100]:
    print "diabetes",alpha, lasso_k_fold_r_squared(x_df,diabetes_df,4, alpha)

LASSO
diabetes 0.001 -5.54087100862
diabetes 0.01 -1.35528362106
diabetes 0.05 0.150816356828
diabetes 0.1 0.296517146933
diabetes 0.5 0.224356968877
diabetes 1 0.040731917599
diabetes 10 -0.0223645436324
diabetes 100 -0.0223645436324


## Ridge 

Again, in this section we'd like to try to use Ridge regression to improve the cross-validated $r^2$ for our model and reduce overfitting of the model.
Below is a function we used to calculate the cross-validated r^2 for ridge regression over a number of folds for a certain parameter value.

In [108]:
from sklearn.linear_model import Ridge
def ridge_k_fold_r_squared(x_train, y_train, num_folds, param_val):
    n_train = x_train.shape[0]
    n = int(np.round(n_train * 1. / num_folds)) # points per fold

    # Iterate over folds
    cv_r_squared = 0
    
    for fold in range(1, num_folds + 1):
        # Take k-1 folds for training 
        x_first_half = x_train.iloc[:n * (fold - 1), :]
        x_second_half = x_train.iloc[n * fold + 1:, :]
        x_train_cv = np.concatenate((x_first_half, x_second_half), axis=0)
        
        y_first_half = y_train.iloc[:n * (fold - 1)]
        y_second_half = y_train.iloc[n * fold + 1:]
        y_train_cv = np.concatenate((y_first_half, y_second_half), axis=0)
        
        # Take the middle fold for testing
        x_test_cv = x_train.iloc[1 + n * (fold - 1):n * fold, :]
        y_test_cv = y_train.iloc[1 + n * (fold - 1):n * fold]

        # Fit Decision Tree model with parameter value on CV train set, and evaluate CV test performance
        reg = Ridge(alpha = param_val, normalize=True)
        reg.fit(x_train_cv, y_train_cv)
        r_squared = reg.score(x_test_cv, y_test_cv)
    
        # Cummulative R^2 value across folds
        cv_r_squared += r_squared

    # Return average R^2 value across folds
    return cv_r_squared * 1.0 / num_folds

### Cardiovascular Diseases
Running Ridge regression on different values of alpha from .001 to 100 yields a top cross-validated $r^2$ score of .425, for alpha = 1.1

In [116]:
print "RIDGE"
for alpha in [.001, .01, .1, .9,1,1.1,1.3,1.5,1.9,5, 10, 11, 100]:
    print "cardio",alpha, ridge_k_fold_r_squared(x_df,cardio_df,5, alpha)

RIDGE
cardio 0.001 -3.96732782949
cardio 0.01 -2.04659444672
cardio 0.1 -0.0743876496175
cardio 0.9 0.415084979043
cardio 1 0.420913047131
cardio 1.1 0.425246636552
cardio 5 0.373288437629
cardio 10 0.285504207166
cardio 11 0.272129846453
cardio 100 0.0449116184953


### Diabetes

Running Lasso on different values of alpha from .001 to 100 yields a top cross-validated $r^2$ score of .317, for alpha = 1

In [122]:
print "RIDGE"
for alpha in [.001, .01, .1, .9,1,3, 5, 10,100, 1000]:
    print "diabetes",alpha, ridge_k_fold_r_squared(x_df,diabetes_df,4, alpha)

RIDGE
diabetes 0.001 -5.5597290689
diabetes 0.01 -2.50308003386
diabetes 0.1 -0.259201378475
diabetes 0.9 0.311280050791
diabetes 1 0.316918281716
diabetes 3 0.316487298488
diabetes 5 0.289080099691
diabetes 10 0.234159110602
diabetes 100 0.0430740924706
diabetes 1000 -0.0145201142658


### Cancer

Running Lasso on different values of alpha from .001 to 100 yields a top cross-validated $r^2$ score of .18, for alpha = 5

In [125]:
print "RIDGE"

for alpha in [.001, .01, .1, 1, 3, 5,10, 20, 1000]:
    print "cancer",alpha, ridge_k_fold_r_squared(x_df,cancer_df,5, alpha)

RIDGE
cancer 0.001 -7.5846526988
cancer 0.01 -3.04209158397
cancer 0.1 -0.743910830295
cancer 1 0.078166346437
cancer 3 0.179617238894
cancer 5 0.182894051905
cancer 10 0.157128814375
cancer 20 0.109819148041
cancer 1000 -0.0589604088889


---
## Regression Trees
---

In this section, we'll try regression trees to see if they improve our model.

Below, we have a function that we'll use to find the $r^2$ value for a given number of folds and certain hyperparameter.

In [129]:
from sklearn.tree import DecisionTreeRegressor
def rtree_k_fold_r_squared(x_train, y_train, num_folds, param_val):
    n_train = x_train.shape[0]
    n = int(np.round(n_train * 1. / num_folds)) # points per fold

    # Iterate over folds
    cv_r_squared = 0
    
    for fold in range(1, num_folds + 1):
        # Take k-1 folds for training 
        x_first_half = x_train.iloc[:n * (fold - 1), :]
        x_second_half = x_train.iloc[n * fold + 1:, :]
        x_train_cv = np.concatenate((x_first_half, x_second_half), axis=0)
        
        y_first_half = y_train.iloc[:n * (fold - 1)]
        y_second_half = y_train.iloc[n * fold + 1:]
        y_train_cv = np.concatenate((y_first_half, y_second_half), axis=0)
        
        # Take the middle fold for testing
        x_test_cv = x_train.iloc[1 + n * (fold - 1):n * fold, :]
        y_test_cv = y_train.iloc[1 + n * (fold - 1):n * fold]

        # Fit Decision Tree model with parameter value on CV train set, and evaluate CV test performance
        reg = DecisionTreeRegressor(max_depth=param_val)
        reg.fit(x_train_cv, y_train_cv)
        r_squared = reg.score(x_test_cv, y_test_cv)
    
        # Cummulative R^2 value across folds
        cv_r_squared += r_squared

    # Return average R^2 value across folds
    return cv_r_squared * 1.0 / num_folds

### Cardiovascular Diseases

For cardiovascular diseases, we see the best $r^2 = .44$ for max_depth = 2. 

In [133]:
for depth in [2, 3, 4, 5, 10, 50, 100]:
    print depth, rtree_k_fold_r_squared(x_df,cardio_df,5, depth)

2 0.440138259697
3 0.375936001262
4 0.261607404615
5 0.315558699673
10 0.160293812685
50 0.218906209133
100 0.232385356672


### Diabetes

For diabetes, we see the best $r^2$ of $.157$ for a max-depth of 2.

In [134]:
for depth in [2, 3, 4, 5, 10, 50, 100]:
    print depth, rtree_k_fold_r_squared(x_df,diabetes_df,5, depth)

2 0.157263311113
3 0.0619619715891
4 -0.0233804974236
5 -0.0805395387135
10 -0.260118795998
50 -0.231097402221
100 -0.257998909033


### Cancer

All of the $r^2$ for decision trees on cancer were negative, indicating that these actually perform worse than the null model and as such are not useful models to examine.

In [139]:
for depth in [2, 3, 5, 8, 9, 10, 20, 50, 70, 100]:
    print depth, rtree_k_fold_r_squared(x_df,cancer_df,5, depth)

2 -0.495027285839
3 -0.382795469727
5 -0.629291531556
8 -0.584912517168
9 -0.598224328667
10 -0.503728749224
20 -0.535395328141
50 -0.566462003329
70 -0.776112364787
100 -0.381191888602
